<a href="https://colab.research.google.com/github/Shatokua/sent_analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ тональности по отношению к выбранному объекту

Конечной целью исследования является отладка модели для определения тональности текста по отношению к выбранному объекту текста, например, к упомянутой в тексте персоне, организации и т.д. 

Такой анализ является более точным, чем стандартный анализ тональности, и представляет больший интерес для конечных пользователей, поскольку дает оценку отношения к конкретному объекту, а не ко всему тексту.

## Подготовка окружения

In [1]:
%%capture
#@title Установка окружения

!pip install -q sklearn==0.22.2.post1
!pip install mendelai-brat-parser==0.0.4
!pip install smart_open==5.1.0
!pip install tensorflow-text==2.5.0


In [2]:
%%capture
#@title Импорт библиотек

import numpy as np
import pandas as pd
import os

from shutil import copyfile
from brat_parser import get_entities_relations_attributes_groups

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # для загрузки universal-sentence-encoder-cmlm/multilingual-preprocess
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [3]:
%%capture
#@title Определение рабочих директорий с данными
SOURCE_DIR1 = 'done/'
SOURCE_DIR2 = 'done1/'
BASE_DIR = 'train_test/'
main_csv_file = 'sent_quotes_done.csv'

## Изучение и подготовка данных

Данные представлены в виде текстовых файлов и файлов аннотаций в формате BRAT.
Все файлы находятся в двух папках done и done1. Файлы внутри папок не разделены по классам. Если в одном файле упоминается несколько объектов, по отношению к которым определяется тональность, данные находятся в одном файле ann.

Для работы в Tensorflow датасет может быть представлен в двух форматах:
1) как структурированный файл (например, csv), где каждая колонка является либо признаком, либо меткой класса;
2) набор файлов, распределенных по директориям-классам. 

Кроме того, необходимо создать копии

Подготовим данные для работы с Tensorflow
(**для последующей работы с Google Colab в дальнейшем будет использоваться сохраненный файл csv, исполнение следующих двух ячеек не требуется**).

In [ ]:
%%capture
#@title Функция для перевода данных в читаемый TensorFlow формат

def files_to_df(source_dir, target_dir):
    """Собираем список файлов txt. 
    Идём по списку файлов, для каждого файла извлекаем текст сообщения (из txt), 
    текст объекта (ann), его индексы (ann), тональность (ann). 
    Раскладываем файлы на папки по классам, параллельно записываем в 
    dataframe pandas"""

    if (target_dir[:-1] not in os.listdir()):
      os.mkdir(target_dir)

    file_names = [fn[:-4] for fn in os.listdir(source_dir) if fn[-3:]=='txt']
    
    text_to_process = []
    
    for fn in file_names:
        txt_file_path = source_dir + fn +'.txt'
        
        with open(txt_file_path, encoding="utf8") as f:
            txt = f.read()
        
        ann_file_path = source_dir + fn +'.ann'
        entities, relations, attributes, groups = get_entities_relations_attributes_groups(ann_file_path)
        entities_keys = list(entities.keys())
        
        for key in entities_keys:
            class_dir = entities[key].type
            if (class_dir not in os.listdir(target_dir)):
                os.mkdir(target_dir + class_dir)
                
            entity_id = entities[key].id
            print(type(entities[key].span[0][0]))

            text_to_process.append({'filename':fn+'_'+entity_id, 'text': txt, 'entity_id': entities[key].id, 
                                    'entity_text': entities[key].text, 'entity_span_start': entities[key].span[0][0],
                                    'entity_span_end': entities[key].span[0][1], 'label': entities[key].type})
            
            txt_new_file_path = target_dir + class_dir + '/' + fn + '_' + entity_id +'.txt'
            copyfile(txt_file_path, txt_new_file_path)
            ent_new_file_path = target_dir + class_dir + '/' + fn + '_' + entity_id + '_entity' +'.txt'
            with open(ent_new_file_path, mode ='w', encoding ="utf8") as f:
              f.write(entities[key].text)
    
    df = pd.DataFrame.from_dict(text_to_process)
    return df


In [4]:
%%capture
#@title Изменение структуры папок, перенос данных в csv

df1 = files_to_df(SOURCE_DIR1, BASE_DIR)
df2 = files_to_df(SOURCE_DIR2, BASE_DIR)
df = pd.concat([df1, df2])
df.to_csv(main_csv_file)

NameError: ignored

Для работы в Google Colab требуется загрузка csv-файла.

In [5]:
%%capture
#@title Чтение файл csv

csv_file = main_csv_file
dataframe = pd.read_csv(csv_file)

Изучим распределение датасета по классам

In [ ]:
%%capture
#@title Распределение классов


In [ ]:
print(dataframe.value_counts('label'))

label
Neutral_all     29070
Neutral         10354
Negative         4836
Positive         4657
Negative_all     1782
Positive_all      923
Mixed             291
Mixed_all          15
dtype: int64


Классы несбалансированы. Класс Mixed_all состоит всего из 15 экземпляров. Укрупним классы, объединив классы с постфиксом _all с одноименными без префикса.
Также приведем метки классов к числовым категориям

In [6]:
%%capture
#@title Объединение классов, приведение меток классов к категориальному формату

classes = {'Positive': 0, 'Positive_all': 0, 'Negative': 1, 'Negative_all': 1,  
           'Mixed': 2, 'Mixed_all': 2, 'Neutral': 3, 'Neutral_all': 3}
dataframe['label'] = dataframe['label'].apply(lambda x: classes[x])

Разделим датасет на набор данных для тренировки и для тестирования.
В тренировочном датасете выделим набор для валидации

In [7]:
%%capture
#@title Объединение классов, приведение меток классов к категориальному формату

train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)


In [ ]:
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

33233 train examples
8309 validation examples
10386 test examples


Поскольку классы не сбалансированы, проверим распределение по классам внутри тренировочного, тестового и валидационного датасетов

In [ ]:
%%capture
#@title Проверка распределения классов


In [8]:
print('Нормализованное распределение по классам в начальном датасете')
print(dataframe.value_counts('label', normalize=True))

print('Нормализованное распределение по классам в датасете train')
print(train.value_counts('label', normalize=True))

print('Нормализованное распределение по классам в датасете test')
print(test.value_counts('label', normalize=True))

print('Нормализованное распределение по классам в датасете val')
print(val.value_counts('label', normalize=True))

Нормализованное распределение по классам в начальном датасете
label
3    0.759205
1    0.127446
0    0.107456
2    0.005893
dtype: float64
Нормализованное распределение по классам в датасете train
label
3    0.760539
1    0.125718
0    0.107724
2    0.006018
dtype: float64
Нормализованное распределение по классам в датасете test
label
3    0.760062
1    0.128538
0    0.105142
2    0.006258
dtype: float64
Нормализованное распределение по классам в датасете val
label
3    0.752798
1    0.132988
0    0.109279
2    0.004934
dtype: float64


Распределение классов одинаково во всех наборах.

## Выбор модели энкодера


Для работы выбрана модель [LaBSE](https://tfhub.dev/google/LaBSE/2) (Language-agnostic BERT sentence embedding model) как одна из наиболее актуальных моделей, показывающих хорошие результаты для русского языка.

Данные для этого энкодера должны быть предварительно обработаны препроцессором [universal-sentence-encoder-cmlm/multilingual-preprocess](https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2)

In [13]:
def get_model(model_url, max_seq_length):
  labse_layer = hub.KerasLayer(model_url, trainable=True)

  # Define input.
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                         name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                     name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="segment_ids")

  # LaBSE layer.
  pooled_output,  _ = labse_layer([input_word_ids, input_mask, segment_ids])

  # The embedding is l2 normalized.
  pooled_output = tf.keras.layers.Lambda(
      lambda x: tf.nn.l2_normalize(x, axis=1))(pooled_output)

  # Define model.
  return tf.keras.Model(
        inputs=[input_word_ids, input_mask, segment_ids],
        outputs=pooled_output), labse_layer


labse_model, labse_layer = get_model(
    model_url="https://tfhub.dev/google/LaBSE/1", max_seq_length=256)

## Изменение стандартного препроцессора

Стандартный препроцессор получает на вход текст и выдает в качестве output словарь из трех тензоров:  
- `input_word_ids`: id поданных на вход слов  
- `input_mask`: маска из 1 и 0, где 1 находится на позициях значимых слов, 0 на позициях паддинга  
- `input_type_ids`: маска из 0 и 1 для передачи дополнительной информации о токенах

Нам необходимо изменить препроцессор таким образом, что в маске `input_type_ids` на позициях токенов интересующего нас объекта стояли 1, на всех остальных позициях - 0.

In [9]:
!pip install bert-for-tf2

     |████████████████████████████████| 41 kB 88 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=87174e32176b1f4a6beaaf97e8e8367c9924cb80da11aa85043ab2ce03cd5e34
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=9a8d86a3acee0bb09d023d2748e4ef3ba3f93c785030930e45ba54acd9fb6a4f
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=3185dc85279213e6aa0390befd9ab217ddc7b812076faa4f9fb9c9dda6bc36a6
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params


[1, 2, 3, 4]

In [43]:
import bert

vocab_file = labse_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = labse_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def create_input(input_strings, entity_starts, entity_ends, tokenizer, max_seq_length):

  input_ids_all, input_mask_all, segment_ids_all = [], [], []
  for i in range (len(input_strings)):
    #Tokenize input calculating entity start and end positions
    input_string = input_strings[i]
    entity_start = entity_starts[i]
    entity_end = entity_ends[i]
    input_tokens_before_entity = ["[CLS]"] + tokenizer.tokenize(input_string[:entity_start])
    input_ids_before_entity = tokenizer.convert_tokens_to_ids(input_tokens_before_entity)
    before_entity_length = len(input_ids_before_entity)
    input_tokens_entity = tokenizer.tokenize(input_string[entity_start:entity_end])
    input_ids_entity = tokenizer.convert_tokens_to_ids(input_tokens_entity)
    entity_length = len(input_ids_entity)
    input_tokens_after_entity = tokenizer.tokenize(input_string[entity_end:])
    input_ids_after_entity = tokenizer.convert_tokens_to_ids(input_tokens_after_entity)
    after_entity_length = len(input_ids_after_entity)

    input_ids = input_ids_before_entity + input_ids_entity + input_ids_after_entity
    sequence_length = before_entity_length+entity_length+after_entity_length

    # Padding or truncation.
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

    segment_ids = [0] * before_entity_length + [1] * entity_length + [0] * after_entity_length
    if(len(segment_ids)>max_seq_length):
      segment_ids = segment_ids[:max_seq_length]
    else:
      segment_ids = segment_ids + [0]*(max_seq_length - len(segment_ids))

    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    segment_ids_all.append(segment_ids)

  return np.array(input_ids_all), np.array(input_mask_all), np.array(segment_ids_all)

def encode(input_text, entity_start, entity_end):
  input_ids, input_mask, segment_ids = create_input(
    input_text, entity_start, entity_end, tokenizer, 256)
  return labse_model([input_ids, input_mask, segment_ids])



In [44]:
test_text = dataframe['text'].to_list()[:5]
test_start = dataframe['entity_span_start'].to_list()[:5]
test_end = dataframe['entity_span_end'].to_list()[:5]
print(test_text)
print(test_start)
print(test_end)

['"да, я допустил неточность, а посол Мюррей ссылается на слухи... но зато теперь мы накопали много РЕАЛЬНОЙ информации на Усманова', '"да, я допустил неточность, а посол Мюррей ссылается на слухи... но зато теперь мы накопали много РЕАЛЬНОЙ информации на Усманова', '"Тот персонаж, о котором вы упомянули [Навальный]..., это тот человек, кого они[американская администрация] хотели бы продвинуть в политическую сферу России и видеть в руководстве страны", заметил Путин, подчеркнув, что США "в этом смысле прокололись', '"Тот персонаж, о котором вы упомянули [Навальный]..., это тот человек, кого они[американская администрация] хотели бы продвинуть в политическую сферу России и видеть в руководстве страны", заметил Путин, подчеркнув, что США "в этом смысле прокололись', '"Хотелось бы знать, Владимир Владимирович, сколько еще на своем посту пробудет губернатор Дарькин, который весь этот беспредел контролирует от начала и до конца и, мягко говоря, набивает карманы?" В ответ Путин пообещал учит

In [45]:
print(encode(test_text, test_start, test_end))

tf.Tensor(
[[ 0.00392828  0.01847563 -0.05777057 ...  0.04605664 -0.07479083
  -0.05366782]
 [ 0.00221529  0.01619829 -0.0579958  ...  0.04617842 -0.07406548
  -0.05412472]
 [-0.04256107  0.01346303 -0.01689823 ... -0.01053682 -0.05535734
  -0.04274627]
 [-0.04247919  0.01359009 -0.01684185 ... -0.01040957 -0.05528823
  -0.04277295]
 [-0.00665717 -0.04999565  0.04001128 ... -0.01140187  0.00798334
  -0.05595917]], shape=(5, 768), dtype=float32)


## Построение модели

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = labse_preprocessor 
  encoder_inputs = preprocessing_layer(text_input)
  encoder = labse_encoder
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.48670408]], shape=(1, 1), dtype=float32)


In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

NameError: ignored